---
### ✅ **ANÁLISIS DEL SECTOR DE INTERNET EN ARGENTINA**
#### *La industria de las telecomunicaciones ha desempeñado un papel crucial en nuestra sociedad, facilitando la información a escala global y permitiendo la comunicación continua. La transferencia de datos y la comunicación se realizan principalmente a través de internet, líneas telefónicas fijas y móviles. Argentina está a la vanguardia en el desarrollo de las telecomunicaciones, contando con un total de 62,12 millones de conexiones en 2020. Dada la relevancia del tema para el país, he llevado a cabo un análisis exhaustivo que permite identificar el comportamiento de este sector a nivel nacional, enfocándome en el acceso al servicio de Internet y su relación con otros servicios de comunicaciones. El objetivo es generar recomendaciones para ofrecer una buena calidad de servicio, identificar oportunidades de crecimiento y plantear soluciones personalizadas para clientes actuales o potenciales.*
---

### 💡 **ETL del Proyecto**

#### *A partir de fuentes de información como el portal de ENACOM (https://indicadores.enacom.gob.ar/datos-abiertos), se obtienen datos asociados al comportamiento histórico trimestral desde el año 2014 hasta el tercer trimestre de 2024 a nivel nacional y, en algunos casos, a nivel provincial. Todos estos datos están concentrados en múltiples hojas dentro de un solo archivo de Excel.*

### ✅ **Carga de archivos Crudos:**

1. **Se inicializa el proceso con la importacion del archivo "Internet.xlsx".**
2. **Con la funcion "split_xls_x_df" creamos un dataframe por cada hoja contenida en el archivo de excel.**
3. **Para facilitar el reconocimiento del contenido de cada campo en los dos dataframe macro se ajustan los nombres de los campos por nombres practicos.**
4. **En una primera exploración de las hojas se idetifica hay datos desagregados por año y trimestre a nivel nacional y hay otros con año, trimestre y provincia por lo cual podríamos reducir la cantidad de dataframes agrupando gran parte de la información en dos macro dataframes (df_Internet_Nacional, df_Internet_Provincias).**
5. **Una vez se han unificado dataframes con campos en comun se hace una limpieza de campos redundantes o repetidos**
6. **Una vez revisado el tipo de información contenida en cada campo, se crean diccionarios con tipos de datos para aplicar un casting y homogenizar la información previendo y mitigando problemas asociados a formato ya que vienen mas validaciones y analisis donde se requiere ya haber depurado estos aspectos**
7. ****


In [55]:
import pandas as pd

# Carga las hojas sin cargar datos, solo los nombres
df_internet = pd.read_excel('/Users/usuario/Documents/M7_LABs_PI/mvp_pi2/data_csv/raw/Internet.xlsx', sheet_name=None)
excel_data = pd.ExcelFile('/Users/usuario/Documents/M7_LABs_PI/mvp_pi2/data_csv/raw/Internet.xlsx')


# Imprime la lista de nombres de hojas
#print(excel_data.sheet_names)

# Ajustar Pandas para que no corte la salida en varias líneas
pd.set_option('display.expand_frame_repr', False)  # Muestra la tabla en una sola línea si la pantalla es ancha
pd.set_option('display.max_columns', None)         # Asegura que se muestren todas las columnas sin truncarlas
pd.set_option('display.width', 1000)               # Ajusta el ancho máximo permitido para la salida

# Validamos las primeras lineas de cada hoja para identifica su contenido
for hojas in df_internet:
    print(f'La hoja {hojas} contiene:') # Imprime el nombre de la hoja
    print(df_internet[hojas].head(4))
    print('\n')

La hoja Acc_vel_loc_sinrangos contiene:
        Partido   Localidad  link Indec Velocidad (Mbps)  Provincia  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.00      1.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.50      2.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.75     19.0
3  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       3.00     85.0


La hoja Velocidad_sin_Rangos contiene:
    Año  Trimestre     Provincia  Velocidad  Accesos
0  2024          2  BUENOS AIRES       75.0     1062
1  2024          2  BUENOS AIRES       59.0       59
2  2024          2  BUENOS AIRES      480.0        5
3  2024          2  BUENOS AIRES        3.5    41735


La hoja Accesos_tecnologia_localidad contiene:
      Provincia     Partido   Localidad    Tecnologia Link Indec  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          ADSL    6854100    755.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo    CABLEMODEM    6854100   4600.

In [ ]:
def renombrar_hojas(df_dict, nuevos_nombres):
    """
    Renombra columnas de las hojas contenidas en df_dict según los renombramientos definidos en nuevos_nombres.
    
    Parámetros:
      - df_dict (dict): Diccionario donde cada clave es el nombre de la hoja y cada valor es un DataFrame.
      - nuevos_nombres (dict): Diccionario de mapeo, donde cada clave es el nombre de la hoja y el valor 
                         es otro diccionario con el mapeo de columnas (llave: nombre original, valor: nuevo nombre).
                         
    Retorna:
      dict: El mismo diccionario df_dict, con las hojas que se encuentran en nuevos_nombres renombradas según lo indicado.
    """
    for hoja, mapping in nuevos_nombres.items():
        if hoja in df_dict:
            df_dict[hoja] = df_dict[hoja].rename(columns=mapping)
        else:
            print(f"La hoja '{hoja}' no se encontró en el diccionario.")
    return df_dict

# Ejemplo de renombramientos para cada hoja
nuevos_nombres = {
    "Totales VMD": {
        "Mbps (Media de bajada)": "Tot_Vel_Media_DL"
    },
    "Totales Accesos Por Tecnología": {
        "ADSL": "Tot_Acc_ADSL",
        "Cablemodem": "Tot_Acc_CaModem",
        "Fibra óptica": "Tot_Acc_FO",
        "Wireless": "Tot_Acc_Wireless",
        "Otros": "Tot_Acc_Otros"
    },
    "Penetracion-totales": {
        "Accesos por cada 100 hogares": "Tot_pntrcion_x_c100_Hoga",
        "Accesos por cada 100 hab": "Tot_pntrcion_x_c100_Habi"
    },
    "Totales Accesos por rango": {
        "Hasta 512 kbps": "Tot_Acc_rango_0_512_kbps",
        "Entre 512 Kbps y 1 Mbps": "Tot_Acc_rango_0.512->1_Mbps",
        "Entre 1 Mbps y 6 Mbps": "Tot_Acc_rango_1->6_Mbps",
        "Entre 6 Mbps y 10 Mbps": "Tot_Acc_rango_6->10_Mbps",
        "Entre 10 Mbps y 20 Mbps": "Tot_Acc_rango_10->20_Mbps",
        "Entre 20 Mbps y 30 Mbps": "Tot_Acc_rango_20->30_Mbps",
        "Más de 30 Mbps": "Tot_Acc_rango_>30_Mbps",
        "OTROS": "Tot_Acc_rango_Otros",
        "Total": "Tot_Acc_rango"
    },
    "Totales Dial-BAf": {
        "Banda ancha fija": "Tot_B_Ancha_Fija",
        "Dial up": "Tot_DialUp",
        "Total": "Tot_DialUp_+_B_Ancha_Fija"
    },
    "Ingresos ": {
        "Ingresos": "Total_Ingresos"
    }
}

# Ejemplo de uso:
# Supongamos que ya tienes el diccionario de DataFrames, por ejemplo:
# df_dict = pd.read_excel("Internet.xlsx", sheet_name=None)
df_internet = renombrar_hojas(df_internet, nuevos_nombres)
# Muestra las columnas renombradas de la hoja "Totales VMD"
print(df_internet["Totales VMD"].head(4))

    Año  Trimestre  Tot_Vel_Media_DL    Trimestre.1
0  2024          2            139.25   Abr-Jun 2024
1  2024          1            139.15   Ene-Mar 2024
2  2023          4            139.04   Oct-Dic 2023
3  2023          3            129.67  Jul-Sept 2023


In [57]:
# Definimos el mapeo para las hojas que contienen información por provincia.
nuevos_nombres_x_provincias = {
    "Velocidad % por prov": {
        "Mbps (Media de bajada)": "Tot_Vel_Media_DL_Trim_x_Prov"
    },
    "Accesos Por Tecnología": {
        "ADSL": "Tot_Acc_ADSL_x_Prov",
        "Cablemodem": "Tot_Acc_CaModem_x_Prov",
        "Fibra óptica": "Tot_Acc_FO_x_Prov",
        "Wireless": "Tot_Acc_Wireless_x_Prov",
        "Otros": "Tot_Acc_Otros_x_Prov",
        "Total": "Tot_Acc_x_Prov"
    },
    "Penetración-poblacion": {
        "Accesos por cada 100 hab": "Tot_pntrcion_x_c100_Habi_x_Prov"
    },
    "Penetracion-hogares": {
        "Accesos por cada 100 hogares": "Tot_pntrcion_x_c100_Hoga_x_Prov"
    },
    "Accesos por rangos": {
        "HASTA 512 kbps": "Tot_Acc_rango_0_512_kbps_x_Prov",
        "+ 512 Kbps - 1 Mbps": "Tot_Acc_rango_0.512->1_Mbps_x_Prov",
        "+ 1 Mbps - 6 Mbps": "Tot_Acc_rango_1->_Mbps_x_Prov",
        "+ 6 Mbps - 10 Mbps": "Tot_Acc_rango_->_Mbps_x_Prov",
        "+ 10 Mbps - 20 Mbps": "Tot_Acc_rango_->_Mbps_x_Prov",
        "+ 20 Mbps - 30 Mbps": "Tot_Acc_rango_->_Mbps_x_Prov",
        "+ 30 Mbps": "Tot_Acc_rango_->_Mbps_x_Prov",
        "OTROS": "Tot_Acc_rango_Otros_Mbps_x_Prov",
        "Total": "Tot_Acc_rango_Mbps_x_Prov"
    },
    "Dial-BAf": {
        "Banda ancha fija": "Tot_B_Ancha_Fija_x_Prov",
        "Dial up": "Tot_DialUp_x_Prov",
        "Total": "Tot_DialUp_+_B_Ancha_Fija_x_Prov"
    }
}

# Ejemplo de uso:
# Supongamos que 'df_dict_prov' es el diccionario de DataFrames para los datos con campo Provincia.
df_internet = renombrar_hojas(df_internet, nuevos_nombres_x_provincias)

In [59]:
# Ajustar Pandas para que no corte la salida en varias líneas
pd.set_option('display.expand_frame_repr', False)  # Muestra la tabla en una sola línea si la pantalla es ancha
pd.set_option('display.max_columns', None)         # Asegura que se muestren todas las columnas sin truncarlas
pd.set_option('display.width', 1000)               # Ajusta el ancho máximo permitido para la salida

# Validamos como quedó el renombramiento de los campos en cada hoja segun se requirió.
for hojas in df_internet:
    print(f'La hoja {hojas} contiene:') # Imprime el nombre de la hoja
    print(df_internet[hojas].head(4))
    print('\n')

La hoja Acc_vel_loc_sinrangos contiene:
        Partido   Localidad  link Indec Velocidad (Mbps)  Provincia  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.00      1.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.50      2.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.75     19.0
3  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       3.00     85.0


La hoja Velocidad_sin_Rangos contiene:
    Año  Trimestre     Provincia  Velocidad  Accesos
0  2024          2  BUENOS AIRES       75.0     1062
1  2024          2  BUENOS AIRES       59.0       59
2  2024          2  BUENOS AIRES      480.0        5
3  2024          2  BUENOS AIRES        3.5    41735


La hoja Accesos_tecnologia_localidad contiene:
      Provincia     Partido   Localidad    Tecnologia Link Indec  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          ADSL    6854100    755.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo    CABLEMODEM    6854100   4600.

In [ ]:
# Revisamos como quedaron todos los campos enombrados en cada una de las hojas del df
for hojas in df_internet:
    print(f'La hoja {hojas} contiene los campos:') # Imprime el nombre de la hoja
    #print(df_internet[hojas].head(4))
    print(df_internet[hojas].columns.tolist())
    print('\n')

La hoja Acc_vel_loc_sinrangos contiene los campos:
['Partido', 'Localidad', 'link Indec', 'Velocidad (Mbps)', 'Provincia', 'Accesos']


La hoja Velocidad_sin_Rangos contiene los campos:
['Año', 'Trimestre', 'Provincia', 'Velocidad', 'Accesos']


La hoja Accesos_tecnologia_localidad contiene los campos:
['Provincia', 'Partido', 'Localidad', 'Tecnologia', 'Link Indec', 'Accesos']


La hoja Velocidad % por prov contiene los campos:
['Año', 'Trimestre', 'Provincia', 'Mbps (Media de bajada)']


La hoja Totales VMD contiene los campos:
['Año', 'Trimestre', 'Mbps (Media de bajada)', 'Trimestre.1']


La hoja Totales Accesos Por Tecnología contiene los campos:
['Año', 'Trimestre', 'ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'Otros', 'Total', 'Periodo']


La hoja Accesos Por Tecnología contiene los campos:
['Año', 'Trimestre', 'Provincia', 'ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'Otros', 'Total']


La hoja Penetración-poblacion contiene los campos:
['Año', 'Trimestre', 'Provincia

In [ ]:
import pandas as pd

def fusionar_por_campo(campo_id, lista_hojas, df_dict):
    """
    Fusiona las hojas indicadas en 'lista_hojas' usando el campo único 'campo_id' como llave de unión.
    
    Parámetros:
      - campo_id (str): Nombre del campo único que se utilizará para fusionar (ej. "id_año_trimestre").
      - lista_hojas (list): Lista de nombres de las hojas a fusionar.
      - df_dict (dict): Diccionario con los DataFrames (clave: nombre de la hoja, valor: DataFrame).
      
    Retorna:
      DataFrame: Resultado de la fusión externa de los DataFrames indicados.
    """
    df_fusionado = None
    
    for hoja in lista_hojas:
        if hoja not in df_dict:
            print(f"La hoja '{hoja}' no se encuentra en el diccionario. Se omite.")
            continue
        df = df_dict[hoja]
        if campo_id not in df.columns:
            print(f"La hoja '{hoja}' no contiene el campo '{campo_id}'. Se omite.")
            continue
        
        if df_fusionado is None:
            df_fusionado = df.copy()
        else:
            df_fusionado = pd.merge(df_fusionado, df, on=campo_id, how='outer', suffixes=('', '_dup'))
    
    if df_fusionado is None:
        return pd.DataFrame()
    return df_fusionado

# Ejemplo de uso:
# Supongamos que 'df_internet' es el diccionario obtenido con pd.read_excel(..., sheet_name=None)
# Fusionar hojas usando el campo "id_año_trimestre"
df_resultado = fusionar_por_campo("id_año_trimestre", ["Velocidad_sin_Rangos", "Totales VMD", "Ingresos"], df_internet)

In [36]:
# Se identifica varias hojas tienen en comun los campos "Año" y "Trimestre", y otras hojas "Año", "Trimestre" y "Provincia"
# Se procede a crear un campo "id_año_trimestre" en cada hoja que contiene esos campos en comun y la ubicamos como primer campo
# Recorremos cada hoja y verificamos si contiene las columnas 'Año' y 'Trimestre'
for hoja, df in df_internet.items():
    if 'Año' in df.columns and 'Trimestre' in df.columns:
        df['id_año_trimestre'] = df['Año'].astype(str) + '-' + df['Trimestre'].astype(str)
        cols = df.columns.tolist()
        cols.remove('id_año_trimestre')
        df = df[['id_año_trimestre'] + cols]
        df_internet[hoja] = df

# Enlistamos todos los campos contenidos en cada una de las hojas del df
for hojas in df_internet:
    print(f'La hoja {hojas} contiene los campos:') # Imprime el nombre de la hoja
    #print(df_internet[hojas].head(4))
    print(df_internet[hojas].columns.tolist())
    print('\n')

La hoja Acc_vel_loc_sinrangos contiene los campos:
['Partido', 'Localidad', 'link Indec', 'Velocidad (Mbps)', 'Provincia', 'Accesos']


La hoja Velocidad_sin_Rangos contiene los campos:
['id_año_trimestre', 'Año', 'Trimestre', 'Provincia', 'Velocidad', 'Accesos']


La hoja Accesos_tecnologia_localidad contiene los campos:
['Provincia', 'Partido', 'Localidad', 'Tecnologia', 'Link Indec', 'Accesos']


La hoja Velocidad % por prov contiene los campos:
['id_año_trimestre', 'Año', 'Trimestre', 'Provincia', 'Mbps (Media de bajada)']


La hoja Totales VMD contiene los campos:
['id_año_trimestre', 'Año', 'Trimestre', 'Mbps (Media de bajada)', 'Trimestre.1']


La hoja Totales Accesos Por Tecnología contiene los campos:
['id_año_trimestre', 'Año', 'Trimestre', 'ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'Otros', 'Total', 'Periodo']


La hoja Accesos Por Tecnología contiene los campos:
['id_año_trimestre', 'Año', 'Trimestre', 'Provincia', 'ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'O

In [ ]:
# Modifico el nombre de los campos para hacerlos mas intuitivos
df_internet['Internet'].rename(columns={'Año':'año', 'Trimestre':'trimestre', 'Provincia':'provincia', 'Hogares con acceso a Internet':'hogares

In [46]:
def obtener_hojas_validas(lista_campos, df_dict, hojas_excluir=None):
    """
    Valida y crea una lista de nombres de hojas que contienen todos los campos indicados en 'lista_campos'.
    
    Parámetros:
      - lista_campos (list): Lista de nombres de campos requeridos.
      - df_dict (dict): Diccionario con DataFrames (clave: nombre de la hoja, valor: DataFrame).
      - hojas_excluir (list): Lista de nombres de hojas a excluir de la validación (puede estar vacía).
    
    Retorna:
      - list: Lista de nombres de hojas que cumplen con tener todos los campos requeridos.
    """
    if hojas_excluir is None:
        hojas_excluir = []
    
    hojas_validas = []
    
    for hoja, df in df_dict.items():
        # Omitimos las hojas que están en la lista de exclusión.
        if hoja in hojas_excluir:
            continue
        
        # Validamos que la hoja contenga todos los campos requeridos.
        if all(campo in df.columns for campo in lista_campos):
            hojas_validas.append(hoja)
    
    return hojas_validas

# Ejemplo de uso:
# Supongamos que 'df_internet' es tu diccionario de DataFrames y deseas obtener las hojas que contienen los campos "Año" y "Trimestre".
campos_ano_trim_prov = ["Año", "Trimestre","Provincia"]
hojas_a_excluir = []

hojas_ano_trim_prov = obtener_hojas_validas(campos_ano_trim_prov, df_internet, hojas_a_excluir)
print("Hojas que contienen los campos año, trimestre y provincia:\n", hojas_ano_trim_prov)
print("\nTotal de hojas encontradas:", len(hojas_ano_trim_prov))

campos_ano_trim = ["Año", "Trimestre"]
hojas_ano_trim = obtener_hojas_validas(campos_ano_trim, df_internet, hojas_ano_trim_prov)
print("Hojas que contienen los campos año y trimestre solamente:\n", hojas_ano_trim)
print("\nTotal de hojas encontradas:", len(hojas_ano_trim))

Hojas que contienen los campos año, trimestre y provincia:
 ['Velocidad_sin_Rangos', 'Velocidad % por prov', 'Accesos Por Tecnología', 'Penetración-poblacion', 'Penetracion-hogares', 'Accesos por rangos', 'Dial-BAf']

Total de hojas encontradas: 7
Hojas que contienen los campos año y trimestre solamente:
 ['Totales VMD', 'Totales Accesos Por Tecnología', 'Penetracion-totales', 'Totales Accesos por rango', 'Totales Dial-BAf', 'Ingresos ']

Total de hojas encontradas: 6


In [ ]:
import pandas as pd

def fusionar_por_campo(campo_id, lista_hojas, df_dict):
    """
    Fusiona las hojas indicadas en 'lista_hojas' usando el campo único 'campo_id' como llave de unión.
    
    Parámetros:
      - campo_id (str): Nombre del campo único que se utilizará para fusionar (ej. "id_año_trimestre").
      - lista_hojas (list): Lista de nombres de las hojas a fusionar.
      - df_dict (dict): Diccionario con los DataFrames (clave: nombre de la hoja, valor: DataFrame).
      
    Retorna:
      DataFrame: Resultado de la fusión externa de los DataFrames indicados.
    """
    df_fusionado = None
    
    for hoja in lista_hojas:
        if hoja not in df_dict:
            print(f"La hoja '{hoja}' no se encuentra en el diccionario. Se omite.")
            continue
        df = df_dict[hoja]
        if campo_id not in df.columns:
            print(f"La hoja '{hoja}' no contiene el campo '{campo_id}'. Se omite.")
            continue
        
        if df_fusionado is None:
            df_fusionado = df.copy()
        else:
            df_fusionado = pd.merge(df_fusionado, df, on=campo_id, how='outer', suffixes=('', '_dup'))
    
    if df_fusionado is None:
        return pd.DataFrame()
    return df_fusionado

# Ejemplo de uso:
# Supongamos que 'df_internet' es el diccionario obtenido con pd.read_excel(..., sheet_name=None)
# Fusionar hojas usando el campo "id_año_trimestre"
df_resultado = fusionar_por_campo("id_año_trimestre", ["Velocidad_sin_Rangos", "Totales VMD", "Ingresos"], df_internet)

In [ ]:
"""
import pandas as pd

def unificar_hojas(df_dict):
    
    Unifica hojas de un archivo Excel en dos DataFrames:
    1. "comunes_ano_trimestre": Hojas que tienen los campos "Año" y "Trimestre".
    2. "comunes_ano_trimestre_provincia": Hojas que tienen los campos "Año", "Trimestre" y "Provincia".
    
    La función también genera y reordena el identificador (id_año_trimestre o id_año_trimestre_prov)
    para que quede en la primera posición de cada DataFrame.
    
    Parámetros:
        df_dict (dict): Diccionario donde la llave es el nombre de la hoja y el valor es el DataFrame.
        
    Retorna:
        dict: Diccionario con dos nuevos DataFrames:
            - "comunes_ano_trimestre"
            - "comunes_ano_trimestre_provincia"
    
    dfs_ano_trimestre = []
    dfs_ano_trimestre_provincia = []
    
    for hoja, df in df_dict.items():
        # Hojas que contienen "Año" y "Trimestre"
        if all(col in df.columns for col in ['Año', 'Trimestre']):
            # Creamos el identificador si no existe
            if 'id_año_trimestre' not in df.columns:
                df['id_año_trimestre'] = df['Año'].astype(str) + '-' + df['Trimestre'].astype(str)
            # Reordenamos para ubicar "id_año_trimestre" en la primera posición
            cols = df.columns.tolist()
            cols.remove('id_año_trimestre')
            df = df[['id_año_trimestre'] + cols]
            dfs_ano_trimestre.append(df)
        
        # Hojas que contienen "Año", "Trimestre" y "Provincia"
        if all(col in df.columns for col in ['Año', 'Trimestre', 'Provincia']):
            if 'id_año_trimestre_prov' not in df.columns:
                df['id_año_trimestre_prov'] = (
                    df['Año'].astype(str) + '-' + 
                    df['Trimestre'].astype(str) + '-' + 
                    df['Provincia'].astype(str).str.upper()
                )
            # Reordenamos para ubicar "id_año_trimestre_prov" en la primera posición
            cols = df.columns.tolist()
            cols.remove('id_año_trimestre_prov')
            df = df[['id_año_trimestre_prov'] + cols]
            dfs_ano_trimestre_provincia.append(df)
    
    # Unificamos (concatenamos) los DataFrames encontrados
    comunes_ano_trimestre = pd.concat(dfs_ano_trimestre, ignore_index=True, sort=False) if dfs_ano_trimestre else pd.DataFrame()
    comunes_ano_trimestre_provincia = pd.concat(dfs_ano_trimestre_provincia, ignore_index=True, sort=False) if dfs_ano_trimestre_provincia else pd.DataFrame()
    
    return {
         "comunes_ano_trimestre": comunes_ano_trimestre,
         "comunes_ano_trimestre_provincia": comunes_ano_trimestre_provincia
    }

# Ejemplo de uso:
# Suponiendo que ya tienes el diccionario df_internet con cada hoja cargada:
unificados = unificar_hojas(df_internet)
df_comunes = unificados["comunes_ano_trimestre"]
df_comunes_prov = unificados["comunes_ano_trimestre_provincia"]
df_comunes.info()
"""